In [ ]:
!git clone https://github.com/mseaberg/lcls_beamline_toolbox #https://github.com/aashwinmishra/lcls_beamline_optimization

In [ ]:
import os
os.chdir('lcls_beamline_toolbox')
!python3 -m pip install -e .
!pip install xraydb -q
!pip install pymoo -q

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lcls_beamline_toolbox.xraywavetrace.beam1d as beam
import lcls_beamline_toolbox.xraywavetrace.optics1d as optics
import lcls_beamline_toolbox.xraywavetrace.beamline1d as beamline
import scipy.optimize as optimize
import copy
import scipy.spatial.transform as transform
from split_and_delay import SND

import torch


import warnings
warnings.filterwarnings("ignore")



from pymoo.core.problem import ElementwiseProblem

from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.termination import get_termination

from pymoo.optimize import minimize

from pymoo.indicators.hv import Hypervolume

In [ ]:
def get_snd_outputs(inputs):
  """
  Study 1 Objective function. Takes an [n, 8] dim np array of
  [samples, ( t1_th1, t1_th2, chi1, chi2, t4_th1, t4_th2, chi1, chi2)].
  The entries lie in the uniform unit interval.
  They are scaled to lie in [-100e-6, +100e-6].
  Returns a torch tensor of dim [n, 2] of
  [samples, (do_sum_objective, IP_r_objective)]
  """
  inputs = inputs*200e-6 - 100e-6
  inputs = inputs.reshape(1,-1)
  result = []

  for x in inputs:
    snd = SND(9500)
    x = np.array(x)

    snd.mvr_t1_th1(x[0])
    snd.mvr_t1_th2(x[1])
    snd.mvr_t1_chi1(x[2])#not in study 1
    snd.mvr_t1_chi2(x[3])#not in study 1
    snd.mvr_t4_th1(x[4])
    snd.mvr_t4_th2(x[5])
    snd.mvr_t4_chi1(x[6])#not in study 1
    snd.mvr_t4_chi2(x[7])#not in study 1

    # snd.mvr_t1_th1(x[0])
    # snd.mvr_t1_th2(x[1])
    # snd.mvr_t4_th1(x[2])
    # snd.mvr_t4_th2(x[3])

    snd.propagate_delay()

    dh1 = snd.get_t1_dh_sum()
    dd = snd.get_dd_sum()
    dh4 = snd.get_t4_dh_sum()
    do = snd.get_do_sum()
    my_IP_sum = snd.get_IP_sum()
    my_intensity = dh1 + dd + dh4 + do #+ my_IP_sum

    do_centroid = snd.get_IP_r()
    do_centroid_x = snd.get_IP_cx()
    do_centroid_y = snd.get_IP_cy()



    result.append([(my_intensity)/(62186.2678), np.log(np.abs(do_centroid_x))/(-16.86781883239746), np.log(np.abs(do_centroid_y))/(-17.84674644470215)])
    del snd
  return np.array(result)

In [ ]:
nscan = 201
ndim = 3
inp = np.ones((nscan,8))*0.500
scanner = np.linspace(0,1, nscan)
inp[:, ndim] = scanner
ys = get_snd_outputs(inp).squeeze()
x_scan = scanner*200e-6 - 100e-6
result1, result2, result3 = ys[0], ys[1], ys[2]

fig, axs = plt.subplots(2, figsize=(12,12))
axs[0].plot(x_scan, np.exp(-result2*16.86), 'k')
axs[0].plot(x_scan, np.exp(-result3*17.84), 'r')
axs[0].set_yscale("log")
axs[0].grid()

axs[1].plot(x_scan, result1, 'k')
axs[1].grid()

# axs[2].plot(x_scan, result1*result2,'k')

In [ ]:
plt.figure(figsize=(8,5))

scale1 = 62186.2678
plt.plot(result1, 'r')
plt.plot(np.power(result1, 3), 'k')

plt.show()

In [ ]:
class MyProblem(ElementwiseProblem):
  def __init__(self):
    super().__init__(n_var=8,
                      n_obj=2,
                      n_ieq_constr=0,
                      xl=np.zeros(8),
                      xu=np.ones(8))

  def _evaluate(self, x, out, *args, **kwargs):
    f1, f2, f3 = get_snd_outputs(x).squeeze()

    out["F"] = [-f1, -f2]
    # out["G"] = [g1, g2]

In [ ]:
problem = MyProblem()

In [ ]:
algorithm = NSGA2(
    pop_size=200,
    n_offsprings=50,
    sampling=FloatRandomSampling(),
    crossover=SBX(prob=0.9, eta=15),
    mutation=PM(eta=20),
    eliminate_duplicates=True
)

In [ ]:
termination = get_termination("n_gen", 100)

In [ ]:
res = minimize(problem,
               algorithm,
               termination,
               seed=1,
               save_history=True,
               verbose=True)

In [ ]:
X = res.X
F = res.F

In [ ]:
plt.figure(figsize=(7, 5))
plt.scatter(-F[:, 0], -F[:, 1], s=30, facecolors='none', edgecolors='blue')
plt.xlabel("Intensity")
plt.ylabel("BPE X")
plt.title("Objective Space")
plt.grid()
plt.show()

In [ ]:
X.shape

In [ ]:
for x in X:
  print(get_snd_outputs(x))

In [ ]:
inp = np.ones((8))*0.500
get_snd_outputs(inp)

In [ ]:
ys1, ys2, ys3 = get_snd_outputs(np.ones((1,8))*0.500).squeeze()
print(ys1, ys2, ys3)

In [ ]:
ys = get_snd_outputs(np.zeros((1,8))).squeeze()
print(ys[0].item(), ys[1].item(), ys[2].item())

In [ ]:
def eval_function(input_dict: dict) -> dict:
  x1, x2, x3, x4, x5, x6, x7, x8 = input_dict["x1"], input_dict["x2"], input_dict["x3"], input_dict["x4"], input_dict["x5"], input_dict["x6"], input_dict["x7"], input_dict["x8"]
  Xinp = np.expand_dims(np.array([x1, x2, x3, x4, x5, x6, x7, x8]), axis=0)
  output = get_snd_outputs(Xinp).squeeze()
  f1, f2, f3 =  output[0].item(), output[1].item(), output[2].item()
  del output, Xinp
  return {"f1": f1, "f2": f2, "f3": f3}

In [ ]:
def run_chain(eval_function = eval_function, n_init: int=5, n_steps: int = 60):
  low = 0.01
  high = 0.99
  vocs = VOCS(
    variables = {"x1": [low, high],
                 "x2": [low, high],
                 "x3": [low, high],
                 "x4": [low, high],
                 "x5": [low, high],
                 "x6": [low, high],
                 "x7": [low, high],
                 "x8": [low, high]},
    objectives = {"f1": "MAXIMIZE", "f2": "MAXIMIZE", "f3": "MAXIMIZE"},
  )
  np.random.seed(42)
  gigo = np.random.rand(8)
  evaluator = Evaluator(function=eval_function)
  ref_point = eval_function({"x1": gigo[0], "x2": gigo[1], "x3": gigo[2],
                             "x4": gigo[3], "x5": gigo[4], "x6": gigo[5],
                             "x7": gigo[6], "x8": gigo[7]})
  generator = MOBOGenerator(vocs=vocs, reference_point= ref_point)
  generator.n_monte_carlo_samples = 512
  generator.numerical_optimizer.n_restarts = 80
  X = Xopt(generator=generator, evaluator=evaluator, vocs=vocs)
  X.random_evaluate(n_init)
  for i in range(n_steps):
    print(i)
    X.step()

  y1 = X.generator.data["f1"]
  y2 = X.generator.data["f2"]
  y3 = X.generator.data["f3"]
  y1_maxs = np.maximum.accumulate(y1)
  y2_maxs = np.maximum.accumulate(y2)
  y3_maxs = np.maximum.accumulate(y3)

  del vocs, evaluator, ref_point, generator, X

  return y1, y2, y3 #y1_maxs, y2_maxs, y3_maxs

In [ ]:
def run_ensemble(n_chains: 25, eval_function = eval_function, n_init: int=3, n_steps: int = 50):
  y1s = []
  y2s = []
  y3s = []
  for i in range(n_chains):
    print(f"Chain: {i+1} of {n_chains}")
    y1, y2, y3 = run_chain(eval_function = eval_function, n_init=n_init, n_steps = n_steps)
    y1s.append(y1)
    y2s.append(y2)
    y3s.append(y3)

  return np.array(y1s), np.array(y2s), np.array(y3s)

In [ ]:
result1, result2, result3 = run_chain(n_init=5, n_steps=80)

In [ ]:
plt.plot(result1, 'k')
plt.plot(result2, 'r')
plt.plot(result3, 'g')

In [ ]:
plt.plot(np.exp(-result2*16.86), 'k')
plt.plot(np.exp(-result3*17.84), 'r')
plt.yscale("log")

In [ ]:
!pip install minepy -q

In [ ]:
from minepy import MINE

In [ ]:
low = 0.01
high = 0.99
vocs = VOCS(
    variables = {"x1": [low, high],
                 "x2": [low, high],
                 "x3": [low, high],
                 "x4": [low, high],
                 "x5": [low, high],
                 "x6": [low, high],
                 "x7": [low, high],
                 "x8": [low, high]},
    objectives = {"f1": "MAXIMIZE", "f2": "MAXIMIZE", "f3": "MAXIMIZE"},
)
vocs

In [ ]:
evaluator = Evaluator(function=eval_function)
evaluator

In [ ]:
ref_point = eval_function({"x1": 0.65, "x2": 0.65, "x3": 0.65, "x4": 0.65, "x5": 0.65, "x6": 0.65, "x7": 0.65, "x8": 0.65})
ref_point

In [ ]:
generator = MOBOGenerator(vocs=vocs, reference_point= ref_point)
generator

In [ ]:
generator.n_monte_carlo_samples = 256
generator.numerical_optimizer.n_restarts = 40


X = Xopt(generator=generator, evaluator=evaluator, vocs=vocs)
X.random_evaluate(500)
X.data

In [ ]:
def get_mic(x, y):
  mine = MINE(alpha=0.6, c=15, est="mic_approx")
  mine.compute_score(x, y)
  MIC=mine.mic()
  return MIC

In [ ]:
mics = []
for x in ["x1", "x2", "x3", "x4", "x5", "x6", "x7", "x8"]:
  mic = get_mic(X.generator.data[x], X.generator.data["f2"])
  print(x, ": ",mic)
  mics.append(mic)

In [ ]:
labels = ["t1_th1", "t1_th2", "t1_chi1", "t1_chi2", "t4_th1", "t4_th2", "t4_chi1", "t4_chi2"]
plt.barh(labels, mics)
plt.grid()
plt.xlabel("Maximum Information Coefficient");

In [ ]:
mics = []
for x in ["x1", "x2", "x3", "x4", "x5", "x6", "x7", "x8"]:
  mic = get_mic(X.generator.data[x], X.generator.data["f3"])
  print(x, ": ",mic)
  mics.append(mic)

In [ ]:
labels = ["t1_th1", "t1_th2", "t1_chi1", "t1_chi2", "t4_th1", "t4_th2", "t4_chi1", "t4_chi2"]
plt.barh(labels, mics)
plt.grid()
plt.xlabel("Maximum Information Coefficient");

In [ ]:
mics = []
for x in ["x1", "x2", "x3", "x4", "x5", "x6", "x7", "x8"]:
  mic = get_mic(X.generator.data[x], X.generator.data["f3"])
  print(x, ": ",mic)
  mics.append(mic)
labels = ["t1_th1", "t1_th2", "t1_chi1", "t1_chi2", "t4_th1", "t4_th2", "t4_chi1", "t4_chi2"]
plt.barh(labels, mics)
plt.grid()
plt.xlabel("Maximum Information Coefficient");

In [ ]:
mics = []
for x in ["x1", "x2", "x3", "x4", "x5", "x6", "x7", "x8"]:
  mic = get_mic(X.generator.data[x], X.generator.data["f2"])
  print(x, ": ",mic)
  mics.append(mic)
labels = ["t1_th1", "t1_th2", "t1_chi1", "t1_chi2", "t4_th1", "t4_th2", "t4_chi1", "t4_chi2"]
plt.barh(labels, mics)
plt.grid()
plt.xlabel("Maximum Information Coefficient");

In [ ]:
mics = []
for x in ["x1", "x2", "x3", "x4", "x5", "x6", "x7", "x8"]:
  mic = get_mic(X.generator.data[x], X.generator.data["f1"])
  print(x, ": ",mic)
  mics.append(mic)
labels = ["t1_th1", "t1_th2", "t1_chi1", "t1_chi2", "t4_th1", "t4_th2", "t4_chi1", "t4_chi2"]
plt.barh(labels, mics)
plt.grid()
plt.xlabel("Maximum Information Coefficient");

In [ ]:
for i in range(100):
  print(i)
  X.step()

In [ ]:
X.generator.data

In [ ]:
y1 = X.generator.data["f1"]
y2 = X.generator.data["f2"]

In [ ]:
plt.scatter(y1, y2)

In [ ]:
scale1, scale2 = 62186.2678, 16.801

In [ ]:
plt.figure(figsize=(8,5))

scale1 = 62186.2678
y1_maxs = np.maximum.accumulate(y1)
y2_maxs = np.maximum.accumulate(y2)
plt.plot(y1_maxs*scale1, 'k')
plt.hlines(scale1, 0, len(y1_maxs), linestyles="dotted")


plt.show()

In [ ]:
plt.plot(np.exp(-y2_maxs*scale2), 'r')
plt.yscale("log")

In [ ]:
y2_maxs.shape

In [ ]:
temp = np.stack([y1_maxs, y2_maxs])
temp.shape

In [ ]:
t = [y1_maxs, y2_maxs]
temp = np.array(t)
temp.shape

In [ ]:
model = generator.train_model()

In [ ]:
model

In [ ]:
gigo = np.random.rand(8)
print(gigo)
print(gigo[0], gigo[1])